## Taylor Imhof
## Bellevue University | DSC 540
## Final Project: Milestone 2
## 2/1/2022

### Cleaning/Formatting Flat File Source

#### Requirements: Perform at least five data transformation and/or cleansing steps to flat file data.
#### Goal: Clean dataset at the end of the milestone

In [1]:
# import required libraries
import pandas as pd
import numpy as np

#### (1) Read into flat files (.tsv) to Pandas dataframe

For my flat file(s), I found two from IMDb interfaces that I believed would be useful for analysis. The first tsv file `title_basics.tsv` contained general movie/TV information such as release year, runtime, genre, and title. However, this dataset did not contain any rating metrics so I decided it would be prudent to combine it with another dataset `title_ratings.tsv` provided from the same set of IMDb interfaces. Since both datasets contained a unique identifier variable `tconst`, I felt it would be easy enough to combine these two after reading them into Pandas dataframes.

Reading in the two dataframes was straightforward, as I was able to make use of Pandas' `read_csv()` method, passing in an additional `sep` argument as the files are tab-separated versus the typical comma-separated.

In [2]:
# read in title_basics.tsv and check first five rows to ensure data loaded properly
titles = pd.read_csv('data/title_basics.tsv', sep='\t', na_values = '\\N')
titles.head()

C:\Users\taylo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [3]:
# read in title_ratings.tsv and check first five rows to ensure data loaded properly
ratings = pd.read_csv('data/title_ratings.tsv', sep='\t', na_values='\\N')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1853
1,tt0000002,6.0,241
2,tt0000003,6.5,1620
3,tt0000004,6.0,156
4,tt0000005,6.2,2448


#### (2) Merge two dataframes into one consolidated dataframe

I felt the next step would be to see if I could merge the two dataframes right away on the `tconst` variable. Getting them consolidated into one dataframe would simplify any cleaning/reformatting endeavors.

In [4]:
# merge titles and ratings dataframes on tconst column
movies_with_ratings = pd.merge(titles, ratings, on='tconst')

In [5]:
# check first five rows to ensure data merged properly
movies_with_ratings.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short",5.7,1853
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short",6.0,241
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance",6.5,1620
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short",6.0,156
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short",6.2,2448


#### (3) Check to see how many rows were lost during merge

As shown above, the two columns from the ratings dataframe `averageRating` and `numVotes` were added properly. One thing I felt would be important to check would see how many observations were lost as a result of merging. To do this, I compared the data contained in `pd.shape` for each of the dataframes.

In [6]:
# display total rows for each individual dataframe
print('titles rows:\t{}'.format(titles.shape[0]))
print('ratings rows:\t{}'.format(ratings.shape[0]))
print('combined rows:\t{}'.format(movies_with_ratings.shape[0]))

# display loss of rows after merging
print('loss of rows:\t{}'.format(titles.shape[0] - ratings.shape[0]))

# display percentage of information lost
print('data lost:\t{}'.format((round(((movies_with_ratings.shape[0] - titles.shape[0]) / titles.shape[0] ) * 100,2))))

titles rows:	8650561
ratings rows:	1210369
combined rows:	1210369
loss of rows:	7440192
data lost:	-86.01


So, nearly 7.5 million rows were lost after merging which is resulted in about an 86% reduction of total observations. While I was sad to see so many rows lost, there were still over 1 million available for analysis. Also, depending on how my efforts go working with the API, I might be able to gain back some of these rows.

#### (4) Inspect datatypes to see how values were encoded during the data loading

Next, after reading in data I typically check the `pd.info()` output to see how the variables were coded during the loading. If there are values that I want numerical that are coded as strings (or vice-versa), these columns will need to be converted appropriately. 

In [7]:
# check info for merged dataframe to verify encoded data types
print(movies_with_ratings.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1210369 entries, 0 to 1210368
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   tconst          1210369 non-null  object 
 1   titleType       1210369 non-null  object 
 2   primaryTitle    1210369 non-null  object 
 3   originalTitle   1210369 non-null  object 
 4   isAdult         1210369 non-null  float64
 5   startYear       1210158 non-null  float64
 6   endYear         36246 non-null    float64
 7   runtimeMinutes  871662 non-null   object 
 8   genres          1190599 non-null  object 
 9   averageRating   1210369 non-null  float64
 10  numVotes        1210369 non-null  int64  
dtypes: float64(4), int64(1), object(6)
memory usage: 110.8+ MB
None


It would seem that all of the columns in the `ratings` dataframe were encoded properly and do not need any cleaning. However, `isAdult`, `startYear`, `endYear`, and `runtimeMinutes` were all encoded as string/object values. 

After reviewing the documentation provided at [IMDb Interfaces](https://www.imdb.com/interfaces/), the curators of the datasets encoded missing or null values with a newline character. As such, I felt that a further refining of the dataset would be necessary, as all of the movies would not have a value stored for `endYear`. As such, I decided to first create a subset that contained only movies to see how many rows were remaining (so sufficient analysis could still be conducted).

#### (5) Create subset that contains only movies

In [8]:
# create subset df that contains only movies
only_movies = movies_with_ratings[movies_with_ratings['titleType'].isin(['movie'])]
only_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
340,tt0000502,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100.0,NaN,4.5,14
374,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70.0,"Action,Adventure,Biography",6.1,747
383,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907.0,NaN,90.0,Drama,5.2,16
397,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0.0,1907.0,NaN,NaN,Drama,4.5,23
405,tt0000630,movie,Hamlet,Amleto,0.0,1908.0,NaN,NaN,Drama,3.8,24


In [9]:
# check shape to see how many rows were lost
print(only_movies.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274383 entries, 340 to 1210358
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          274383 non-null  object 
 1   titleType       274383 non-null  object 
 2   primaryTitle    274383 non-null  object 
 3   originalTitle   274383 non-null  object 
 4   isAdult         274383 non-null  float64
 5   startYear       274355 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  246575 non-null  object 
 8   genres          264106 non-null  object 
 9   averageRating   274383 non-null  float64
 10  numVotes        274383 non-null  int64  
dtypes: float64(4), int64(1), object(6)
memory usage: 25.1+ MB
None


This resulted in a much smaller dataframe with only ~274,000 rows. However, I felt this was still quite large and I would still be able to discover interesting insights.

#### (6) Drop redundant columns

Since I am only dealing with movies, the column `endYear` is useless as this field was only used to track data for TV shows that air over a period of time, rather than a release date with movies. Also, there is an `originalTitle` column that I believe tracks if the title was changed or if the title was in another language. I felt that this column could also be dropped without much impact to analysis efforts. Also, since I am only dealing with movies, the `titleType` column could also be dropped safely.

In [10]:
movies_noredun = only_movies.drop(['titleType','originalTitle', 'endYear'], axis=1)
movies_noredun.head()

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
340,tt0000502,Bohemios,0.0,1905.0,100.0,NaN,4.5,14
374,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,"Action,Adventure,Biography",6.1,747
383,tt0000591,The Prodigal Son,0.0,1907.0,90.0,Drama,5.2,16
397,tt0000615,Robbery Under Arms,0.0,1907.0,NaN,Drama,4.5,23
405,tt0000630,Hamlet,0.0,1908.0,NaN,Drama,3.8,24


#### (7) Check For Duplicate Values

In [11]:
# check for count of duplicated values
movies_noredun.duplicated().sum()

0

It does not look like my dataframe at this stage contains any duplicate values!

#### (8) Addressing Missing/Null Values

I felt the next step to take was to handle the missing data. First, I checked for the existence of any null values using the implementation below that checks for null values for each column. It showed that there were NO missing values at all! However, as noted in an earlier step, all of the missing/null values were encoded with a newline character.  

It was during my research for this step, I found that there is a way to encoded specfic characters as Nans during the load stage by passing an additional `na_values` argument to the `pd.read_csv()` method. So I decided to make a copy at this stage and re-attempt my data loading with this new argument. I actually found this to be very useful and actually changed the datatypes of some of the columns I was going to have to change in a future step!

After converting the newline characters to `NaN`s I could accurately assess how many missing values existed in the dataframe using the `sum()` of the `isnull()` method.

In [12]:
# check for NaNs
movies_noredun.isnull().sum()

tconst                0
primaryTitle          0
isAdult               0
startYear            28
runtimeMinutes    27808
genres            10277
averageRating         0
numVotes              0
dtype: int64

In [13]:
# check info to verify datatypes
movies_noredun.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274383 entries, 340 to 1210358
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          274383 non-null  object 
 1   primaryTitle    274383 non-null  object 
 2   isAdult         274383 non-null  float64
 3   startYear       274355 non-null  float64
 4   runtimeMinutes  246575 non-null  object 
 5   genres          264106 non-null  object 
 6   averageRating   274383 non-null  float64
 7   numVotes        274383 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 18.8+ MB


There were only 28 observations with missing `startYear` values, however there were still quite a few rows with missing values for `runtimeMinutes` and `genres`. I thought it would be important to check the datatypes. 

It looks like `runtimeMinutes` was still being encoded as an object. First, I decided to drop all rows with missing values in this column. After doing this, it would seem that all null values were accounted for, as the values in the `Int64Index` and `Non-Null` count for each column were both `240851`.

In [14]:
# drop all rows where runtimeMinutes value was missing
movies_noredun.dropna(axis=0, how='any', inplace=True)
movies_noredun.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240851 entries, 374 to 1210358
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          240851 non-null  object 
 1   primaryTitle    240851 non-null  object 
 2   isAdult         240851 non-null  float64
 3   startYear       240851 non-null  float64
 4   runtimeMinutes  240851 non-null  object 
 5   genres          240851 non-null  object 
 6   averageRating   240851 non-null  float64
 7   numVotes        240851 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 16.5+ MB


In [15]:
# check again for NaNs
movies_noredun.isnull().sum()

tconst            0
primaryTitle      0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
averageRating     0
numVotes          0
dtype: int64

All null values have been accounted for!

#### (9) Convert column to specific datatype

After accounting for missing data, I still had to convert all of the values in `runtimeMinutes` to `float64` so that I could conduct analysis of numerical data. I implemented this using the `pd.astype()` method, passing in a dictionary with a single entry containing the column name to be dropped `runtimeMinutes` and the desired data type to be converted to `float`.

In [16]:
# convert all runtimeMinutes values to float datatype
movies_noredun = movies_noredun.astype({'runtimeMinutes':float})
movies_noredun.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240851 entries, 374 to 1210358
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          240851 non-null  object 
 1   primaryTitle    240851 non-null  object 
 2   isAdult         240851 non-null  float64
 3   startYear       240851 non-null  float64
 4   runtimeMinutes  240851 non-null  float64
 5   genres          240851 non-null  object 
 6   averageRating   240851 non-null  float64
 7   numVotes        240851 non-null  int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 16.5+ MB


In [17]:
movies_noredun.head()

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
374,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,"Action,Adventure,Biography",6.1,747
383,tt0000591,The Prodigal Son,0.0,1907.0,90.0,Drama,5.2,16
438,tt0000679,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,"Adventure,Fantasy",5.3,65
740,tt0001184,Don Juan de Serrallonga,0.0,1910.0,58.0,"Adventure,Drama",3.9,20
769,tt0001258,The White Slave Trade,0.0,1910.0,45.0,Drama,5.7,139


#### (10) Exacting One Genre For Simplicity

Another problem I felt would impact analysis was the fact that the `genres` column could contain multiple values. I felt for simplicity, I could just extract the first genre type as they were all separated using commas. First, I had to use the `split()` method to create a list of strings for each value in the `genres` column. Then, I used the `explode()` method to assign the first string in each list of strings to the only value in the `genres` column. I also decided to reset the index here for consistency.

In [18]:
# check for unique values in the genres column
movies_noredun['genres'].unique()

array(['Action,Adventure,Biography', 'Drama', 'Adventure,Fantasy', ...,
       'Documentary,Family,Sci-Fi', 'Action,Documentary,Mystery',
       'Action,Crime,Short'], dtype=object)

In [19]:
# change genre values to list of strings split on comma
movies_noredun['genres'] = movies_noredun['genres'].str.split(',')

# assign first string from list of strings to column value
movies_one_genre = movies_noredun.explode('genres').reset_index(drop=True)

# check first five rows to ensure changes were implemented appropriately
movies_one_genre.head()

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,Action,6.1,747
1,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,Adventure,6.1,747
2,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,Biography,6.1,747
3,tt0000591,The Prodigal Son,0.0,1907.0,90.0,Drama,5.2,16
4,tt0000679,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,Adventure,5.3,65


It looks like when I split on the genres, duplicate rows were created for each row that had multiple genres. To remedy this, I simply dropped all duplicates of the `primaryTitle` rows, while keeping the first instance of the title.

In [20]:
movies_one_genre['primaryTitle'].duplicated()

0         False
1          True
2          True
3         False
4         False
          ...  
426031     True
426032    False
426033     True
426034     True
426035    False
Name: primaryTitle, Length: 426036, dtype: bool

In [21]:
movies_one_genre.drop_duplicates(subset=['primaryTitle'], keep='first', inplace=True, ignore_index=True)
movies_one_genre.head()

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,Action,6.1,747
1,tt0000591,The Prodigal Son,0.0,1907.0,90.0,Drama,5.2,16
2,tt0000679,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,Adventure,5.3,65
3,tt0001184,Don Juan de Serrallonga,0.0,1910.0,58.0,Adventure,3.9,20
4,tt0001258,The White Slave Trade,0.0,1910.0,45.0,Drama,5.7,139


#### (11) Renaming Columns For Consistency

Now that I have all of my columns in the right data type and missing values have been accounted for, the next thing I wanted to do was change some of the column names so that they were more consistent and in line with what I am used to. The column names were encoded using camel case, though I have been more used to the convention of using underscores for variables with multiple words. Also I felt that I could simplify the wording for some of the columns to make coding a little more streamlined.

In [22]:
# rename columns for convention and easier coding
movies_final = movies_one_genre.rename(columns={'tconst':'id', 
                                              'primaryTitle':'title', 
                                              'isAdult':'is_adult',
                                              'startYear':'release_year',
                                              'runtimeMinutes':'runtime', 
                                              'averageRating':'rating', 
                                              'numVotes':'votes'})
movies_final.head()

,id,title,is_adult,release_year,runtime,genres,rating,votes
0,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,Action,6.1,747
1,tt0000591,The Prodigal Son,0.0,1907.0,90.0,Drama,5.2,16
2,tt0000679,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,Adventure,5.3,65
3,tt0001184,Don Juan de Serrallonga,0.0,1910.0,58.0,Adventure,3.9,20
4,tt0001258,The White Slave Trade,0.0,1910.0,45.0,Drama,5.7,139
